In [ ]:
from tqdm import tqdm
import numpy as np
from copy import deepcopy

## Part 1

In [ ]:
# contains the name of the set of instructions, and a list with all the instructions, whether they are conditional or not
class instruction:
    def __init__(self, name: str, instructions = []):
        self.name = name
        # to be able to generate the two stopping instructions
        if name in 'AR':
            return None
        # generates a list of the length of the instructions
        self.instr = [None for idx in range(len(instructions))]
        # initializes the elements of self.instr
        for idx, instruction in enumerate(instructions[:-1]):
            # if the instruction is uniquely A or R, we report it and leave the initialization
                if instruction in 'AR':
                    self.instr.append(instruction)
                    return None
                # we split the instruction in letter, > or <, value, new instruction
                self.instr[idx] = [instruction[0], instruction[1]]+ instruction[2:].split(':')
                self.instr[idx][2] = int(self.instr[idx][2])
        self.instr[-1] = [instructions[-1]]
        
    # for debugging purposes    
    def __repr__(self):
        return self.name + ' ' + str(self.instr)

In [ ]:
op = {'>': lambda x, y: x > y,
      '<': lambda x, y: x < y}
# each member contains a dict_let with the numbers for x, m, a, s and a cur_op for the current operation/instruction
class parts:
    # initialization with 4 values
    def __init__(self, attributes):
        self.dict_let = {'x' : int(attributes[0][2:]),
                     'm' : int(attributes[1][2:]),
                     'a' : int(attributes[2][2:]),
                     's' : int(attributes[3][2:])
                    }
        self.cur_op = 'in'
    # applies the instruction/operation until we jump to another instruction    
    def apply_op(self, instructions: list[instruction]) -> None:
        for instr in instructions:
            # if the instruction is a forced jump to another instruction, we do it and leave
            if len(instr) == 1:
                self.cur_op = instr[0]
                return None
            # as soon as we have to jump instruction, we update self.cur_op and leave
            if op[instr[1]](self.dict_let[instr[0]],instr[2]):
                self.cur_op = instr[3]
                return None
    # for debugging        
    def __repr__(self):   
        return str(self.dict_let) + ' ' + self.cur_op
     
    # to compute part 1 
    def sum_gears(self):
        return sum(self.dict_let.values()) if self.cur_op == 'A' else 0

    

In [ ]:
instructions = {}
# initializes A and R instructions
instructions['A'] = True
instructions['R'] = False
gears = []
value_accepted_gears = 0

with open('Day19_input.txt') as f:
    are_instructions = True
    for line in tqdm(f):
        line = line.strip(' {}\n').replace('{', ',').split(',')
        
        #separates the instructions from the gears
        if line == ['']:
            are_instructions = False
            continue
        # saves the instructions in the dictionary    
        if are_instructions: 
            instructions[line[0]] = instruction(line[0], line[1:])
        # gears
        else:
            new_gear = parts(line)
            gears.append(new_gear)
            # while we have not reached either A or R, we go through the instructions
            while new_gear.cur_op not in 'AR':
                new_gear.apply_op(instructions[new_gear.cur_op].instr)
            # once we reached A or R, we add the value (only if A)
            value_accepted_gears +=new_gear.sum_gears()
                
print(value_accepted_gears)

## Part 2

In [ ]:
# updated class to contain instead ranges of values
class parts_ranges:
    def __init__(self, old_part_range = None):
        if old_part_range:
            # the [] + ... is to get around the shallow-copy issue of tuples
            self.dict_let = {'x' : [] + old_part_range.dict_let['x'],
                             'm' : [] + old_part_range.dict_let['m'],
                             'a' : [] + old_part_range.dict_let['a'],
                             's' : [] + old_part_range.dict_let['s']
                            }
            self.cur_op = str(old_part_range.cur_op)
        else:
            # if no input given, we have the starting value with the ranges
            self.dict_let = {'x' : [1, 4000],
                             'm' : [1, 4000],
                             'a' : [1, 4000],
                             's' : [1, 4000]
                            }
            self.cur_op = 'in'
     
    # this augmented apply operation tries to follow the instruction until the end, and in the process
    # generates slices of the parts_ranges going to the correct instructions, and returns them as list
    def apply_op(self, instructions: list[instruction]) :
        split_parts_ranges = []
        # for each instruction in the list of instructions
        for instr in instructions:
            # if we see a forced change
            if len(instr) == 1:
                self.cur_op = instr[0]
                return split_parts_ranges
           
            # had to split the > from the < case
            # in the > case, we generate a copy with the top part of that attribute range
            # and append it to the list of splitted parts. The original parts loses that slices,
            # but keeps following the instructions as long as some pieces remain
            if instr[1] == '>':
                # if max of range of letter value is bigger than instr[2]
                if self.dict_let[instr[0]][1] > instr[2]:
                    # create top parts_ranges with the base values of self and split the right way
                    _, top_half = self.__split_gear_in_two(instr[0], instr[2])
                    # gives it the new updated address
                    top_half.cur_op = instr[3]
                    # appends it to the new
                    split_parts_ranges.append(top_half)                    
                    # updates the right letter of self = bottom_half with the amount removed by top_half
                    self.dict_let[instr[0]][1] = min(self.dict_let[instr[0]][1], instr[2])
                    # if the range is empty or negative, we exit
                    if self.dict_let[instr[0]][0] >= self.dict_let[instr[0]][1]:
                        #print('leaving from >\n', self,'\nsplit_part_ranges = ', split_parts_ranges)
                        return split_parts_ranges
            
            # same as with the > part, but now the bottom part gets sent to the list of extra parts
            if instr[1] == '<':
                # if min of range of self is lower than instr[2]
                if self.dict_let[instr[0]][0] < instr[2]:
                    # create bottom parts_ranges with the base values of self and split the right way
                    # NOW with a -1 in the second input!!!
                    bottom_half, _ = self.__split_gear_in_two(instr[0], instr[2] - 1)
                    # gives it the new updated address
                    bottom_half.cur_op = instr[3]
                    # appends it to the new
                    split_parts_ranges.append(bottom_half)
                    
                    # updates the right letter of self = top_half with the amount removed by bottom_half
                    self.dict_let[instr[0]][0] = max(self.dict_let[instr[0]][0], instr[2])
                    # if the range is empty or negative, we exit
                    if self.dict_let[instr[0]][0] >= self.dict_let[instr[0]][1]:
                        #print('leaving from <\n', self,'\nsplit_part_ranges = ', split_parts_ranges)
                        return split_parts_ranges
                    
                    
    # we make two copies of the self gear, and on the attribute letter we split it
    def __split_gear_in_two(self, letter, value):
        bottom_gear = parts_ranges(self)
        # the right value of bottom is value if it's lower than the right value of self
        bottom_gear.dict_let[letter][1] = min(value, bottom_gear.dict_let[letter][1])
        
        top_gear = parts_ranges(self)
        # the left value of top is value+1 if it's bigger than the left value of self
        top_gear.dict_let[letter][0] = max(value+1, top_gear.dict_let[letter][0])
        return bottom_gear, top_gear
    # for debugging purposes                
    def __repr__(self):   
        return str(self.dict_let) + ' - ' + self.cur_op
    # to solve part 2
    def sum_gears(self):
        if self.cur_op == 'A':
            total = 1
            for values in self.dict_let.values():
                total *= (values[1] - values[0]+1)
            return total
        else:
            return 0
    

In [ ]:
instructions = {}
# add basic instructions
instructions['A'] = True
instructions['R'] = False
# add total 1-4000 x 4 part
gears = [parts_ranges()]
value_accepted_gears = 0

with open('Day19_input.txt') as f:
    are_instructions = True
    for line in tqdm(f):
        line = line.strip(' {}\n').replace('{', ',').split(',')
        if line == ['']:
            break
        instructions[line[0]] = instruction(line[0], line[1:])
            
for new_gear in gears:
    # as long as we are not arrived in either A or R
    while new_gear.cur_op not in 'AR':
        # we take part_ranges() gear and run it through the instructions until the main body hits an A or R.
        # the output is the list of part_ranges() that were split along the way
        added_gears = new_gear.apply_op(instructions[new_gear.cur_op].instr)
        gears += added_gears 
    value_accepted_gears +=new_gear.sum_gears()
                
print(value_accepted_gears)
